In [1]:
#14.7.2021

from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np
import cv2
import os
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans
import cv2 
import matplotlib.pyplot as plt
%matplotlib inline

# Import datasets, classifiers and performance metrics
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split


In [2]:
train_path = 'ubiris1_6/train'  # Folder Names are Parasitized and Uninfected
training_names = os.listdir(train_path)

# Get path to all images and save them in a list
# image_paths and the corresponding label in image_paths
image_paths = []
image_classes = []
class_id = 0

#To make it easy to list all file names in a directory let us define a function
#
def imglist(path):
	return [os.path.join(path, f) for f in os.listdir(path)]

#Fill the placeholder empty lists with image path, classes, and add class ID number
for training_name in training_names:
	dir = os.path.join(train_path, training_name)
	class_path = imglist(dir)
	image_paths+=class_path
	image_classes+=[class_id]*len(class_path)
	class_id+=1

# Create feature extraction and keypoint detector objects
    #SIFT is not available anymore in openCV    
# Create List where all the descriptors will be stored
des_list = []

In [3]:
sift = cv2.xfeatures2d.SIFT_create()

for image_path in image_paths:
	im = cv2.imread(image_path)
	img = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
	# Adaptive Gaussian Thresholding
	#th1 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
	# Otsu's thresholding
	#ret2,th2 = cv2.threshold(th1,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
	# Otsu's thresholding after Gaussian filtering
	#blur = cv2.GaussianBlur(th2,(5,5),0)
	#ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
	#img = img.astype('uint8')
	kpts, des = sift.detectAndCompute(img, None)
	des_list.append((image_path, des))
print(des_list)
print("---------------------")
print(des)

[('ubiris1_6/train/c/Img_3_1_4.jpg', array([[ 87.,  56.,   1., ...,   0.,   0.,   2.],
       [  0.,   0.,   0., ...,   0.,   1., 131.],
       [ 46.,  30.,   1., ...,   0.,   0.,  37.],
       ...,
       [  5.,   1.,   1., ...,   0.,   0.,  13.],
       [  0.,   0.,   0., ...,   3.,   1.,   3.],
       [ 32.,   0.,   0., ...,   1.,   6.,  25.]], dtype=float32)), ('ubiris1_6/train/c/Img_3_1_1.jpg', array([[ 2.,  0.,  1., ...,  0.,  0., 15.],
       [41., 21.,  2., ...,  5.,  4.,  1.],
       [98., 11.,  0., ..., 15., 36., 20.],
       ...,
       [ 7.,  9.,  5., ...,  1., 26., 33.],
       [16., 21.,  3., ...,  2.,  3.,  4.],
       [ 0.,  0.,  0., ...,  4.,  6.,  6.]], dtype=float32)), ('ubiris1_6/train/c/Img_3_1_3.jpg', array([[ 27.,   1.,   0., ...,   0.,   2.,  55.],
       [121.,  20.,   0., ...,   5.,   3.,   2.],
       [ 47.,   9.,   0., ...,   0.,   0.,  28.],
       ...,
       [  1.,   0.,   0., ...,   0.,   0.,  52.],
       [ 14.,   1.,   0., ...,   2.,   0.,   0.],
     

In [6]:
des[0]

array([  0.,   0.,   0.,   0.,   6.,  23.,  23.,   2.,  59.,   5.,   0.,
         0.,   6.,  34.,  13.,  13., 101.,  27.,   0.,   0.,   1.,   2.,
         1.,   5.,   2.,   1.,   0.,   0.,   8.,  21.,   4.,   2.,  28.,
        22.,   0.,   0.,   3.,  37.,  65.,  35.,  83.,  21.,   0.,   0.,
        37., 131.,  55.,  27., 131.,  92.,   0.,   0.,   5.,  10.,   5.,
        21.,  29.,   4.,   0.,   0.,   7.,  39.,  26.,  28., 131.,  21.,
         0.,   0.,   9.,  25.,  14.,  80.,  50.,   2.,   0.,   1., 131.,
       131.,  22.,  42., 131.,   5.,   0.,   0.,  16.,  19.,  14., 108.,
        47.,   0.,   0.,   0.,   0.,   3.,  40.,  90.,  79.,  62.,   5.,
         3.,  18.,  26.,   5.,  23.,  35.,  32.,   6.,  12., 119.,  24.,
         8.,  26., 131.,   0.,   0.,   0.,  14.,   6.,   9., 131.,  36.,
         0.,   0.,   0.,   2.,   9.,  41.,  65.], dtype=float32)

In [7]:
des.shape   #that are keypoints, you have to clustering them----> results will be clusters of keypoints---> each cluster is feature

(155, 128)

In [8]:
from sklearn.cluster import KMeans
kmeans = KMeans(init="k-means++", n_clusters=100, n_init=4,random_state=0)
#X=descriptors
kmeans.fit(des)
visual_words = kmeans.cluster_centers_
print(kmeans)
print(visual_words)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=100, n_init=4, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)
[[ 9.         6.799999  15.799999  ...  0.8000002  0.6000004 20.4      ]
 [ 2.         3.        59.        ... 11.        18.        33.       ]
 [71.600006  21.         0.5999994 ...  2.3999996  2.8000002  5.6000004]
 ...
 [32.        22.        38.        ... 29.        22.        47.       ]
 [ 2.        18.        66.        ...  2.        14.        50.       ]
 [ 1.         1.5        3.        ...  3.         8.        74.5      ]]


In [9]:
visual_words.shape   #each cluster is feature---> this=visual_words is test mn.

(100, 128)

In [10]:
visual_words[0]  #first feature, for first image=first cluster

array([  9.        ,   6.799999  ,  15.799999  ,  31.800003  ,
        73.2       ,  28.        ,  19.6       ,   7.2       ,
        69.        ,  30.4       ,   8.6       ,  18.4       ,
        48.        ,  14.2       ,   3.7999997 ,  30.        ,
       128.2       ,  59.4       ,   5.6       ,   3.5999994 ,
         4.        ,   0.6000004 ,   1.        ,  25.2       ,
        78.4       ,  31.6       ,   5.2000003 ,   4.2       ,
        10.799999  ,   6.        ,   1.6000004 ,  11.8       ,
        21.6       ,   7.2       ,  11.8       ,  48.8       ,
        92.        ,  26.4       ,   6.7999997 ,   9.        ,
        49.199997  ,  40.8       ,  24.2       ,  33.6       ,
        61.4       ,  22.        ,   8.        ,  14.        ,
       138.2       ,  85.399994  ,  18.        ,   5.4000006 ,
         5.4000015 ,   1.2000008 ,   0.19999886,  15.200001  ,
       120.600006  ,  30.4       ,   3.        ,   2.8000002 ,
        11.200001  ,   9.8       ,   1.2000008 ,  13.4 

In [11]:
#if (descriptor is not None):https://towardsdatascience.com/bag-of-visual-words-in-a-nutshell-9ceea97ce0fb
histogram = build_histogram(visual_words, kmeans)
preprocessed_image.append(histogram)

NameError: name 'build_histogram' is not defined

In [12]:
from scipy.cluster.vq import kmeans, vq

k = 100  #k means with 100,600,500,300 clusters gives lower accuracy 
voc, variance = kmeans(des, k, 1) 

# Calculate the histogram of features and represent them as vector
#vq Assigns codes from a code book to observations.
im_features = np.zeros((len(image_paths), k), "float32")
for i in range(len(image_paths)):
	words, distance = vq(des_list[i][1],voc)
	for w in words:
		im_features[i][w] += 1

In [13]:
from sklearn.preprocessing import StandardScaler
im_features = StandardScaler().fit_transform(im_features)

In [14]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(200, ), random_state=1, max_iter=100000, activation='logistic')       #, batch_size=400)


In [15]:
# evaluate model using 10-fold cross validation
from sklearn.model_selection import StratifiedKFold, cross_val_score
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)  #crossvalidation for better results, n_splits=10 cannot be greater than the number of members in each class
results = cross_val_score(clf, im_features, np.array(image_classes), cv=kfold)
clf.fit(im_features, np.array(image_classes))

MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(200,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=100000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [16]:
#Joblib dumps Python object into one file
from sklearn.externals import joblib
joblib.dump((clf, training_names, k, voc), "ubiriscnn1_6.pkl", compress=3) #izzvzela stdSlr

/home/alenkat/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['ubiriscnn1_6.pkl']